In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_absolute_error as mae
pd.set_option('display.float_format', '{:.2f}'.format) 

import matplotlib.pyplot as plt
plt.style.use('bmh')
%matplotlib inline

In [3]:
df = pd.read_hdf('../input/property_polish.h5')
df.sample(5)

,building,extra_benefits,parking,apartament,offer,surroundings,extra_description,benefit_investment,broker_license,stats,...,location,finishing,bathroom,apartament_fix,price,y_pred_price_mean,price_m2,y_pred_price_m2_mean_area,price_log,price_log_m2
0,"{'Typ budynku': 'apartamentowiec', 'Liczba pię...",winda,NaN,"{'Liczba pokoi': '5', 'Kuchnia': 'do wykończen...",3 596 050 zł (26 500 zł/m,{'Zdrowie': [' ...,[Maxon Nieruchomości jest zewnętrznym biurem s...,NaN,,"[[Źródło: , Biuro nieruchomości], [Rynek: , wt...",...,-1,-1,-1,"{'Liczba pokoi': '4', 'Kuchnia': 'w aneksie', ...",3596050.00,688142.59,26500.00,1153456.45,15.10,0.11
0,"{'Typ budynku': 'kamienica', 'Liczba pięter': ...",NaN,NaN,"{'Liczba poziomów': '4', 'Powierzchnia': '116 ...",3 400 000 zł (29 310 zł/m,{'Zdrowie': [' ...,[Apartament w stylowej kamienicy w okolicy Pl....,NaN,,"[[Źródło: , Biuro nieruchomości], [Rynek: , wt...",...,-1,-1,-1,"{'Liczba poziomów': 'jednopoziomowe', 'Powierz...",3400000.00,688142.59,29310.34,986005.51,15.04,0.13
0,"{'Liczba pięter': '2', 'Rok budowy': '2006', '...",NaN,NaN,"{'Liczba pokoi': '1', 'Forma własności': 'włas...",998 000 zł (12 321 zł/m,{'Zdrowie': [' ...,[Imielin - elitarne osiedle kameralnych niskic...,NaN,Nr licencji pośrednika: 3536,"[[Źródło: , Biuro nieruchomości], [Rynek: , wt...",...,-1,-1,-1,"{'Liczba pokoi': '3', 'Forma własności': 'włas...",998000.00,688142.59,12320.99,688503.85,13.81,0.17
0,"{'Typ budynku': 'blok', 'Liczba pięter': '6', ...","telefon, winda, domofon, ochrona","miejsce parkingowe, parking podziemny","{'Liczba pokoi': '2', 'Stan łazienki': 'z okne...",1 200 000 zł (8 658 zł/m,{'Zdrowie': [' ...,[Ciche i przestronne mieszkanie na tyłach Blue...,NaN,,"[[Źródło: , Biuro nieruchomości], [Rynek: , wt...",...,"północne, południowe",-1,-1,"{'Liczba pokoi': '5', 'Stan łazienki': 'dobry'...",1200000.00,688142.59,8658.01,1178106.58,14.00,0.10
0,"{'Typ budynku': 'blok', 'Liczba pięter': '6', ...",winda,NaN,"{'Liczba pokoi': '1', 'Liczba poziomów': '3', ...",690 000 zł (9 324 zł/m,{},"[| , \nsłoneczne mieszkanie na pierwszym piętr...",NaN,Nr licencji pośrednika: 3154,"[[Źródło: , Biuro nieruchomości], [Rynek: , wt...",...,-1,-1,-1,"{'Liczba pokoi': '3', 'Liczba poziomów': 'jedn...",690000.00,688142.59,9324.32,629003.52,13.44,0.18


In [4]:
df['y_pred_price_mean'] = df['price'].mean()

In [5]:
mae(df.price, df.y_pred_price_mean)

408331.10340498947

In [6]:
df['price_m2'] = df.price / df.area
global_mean_price_m2 = df.price_m2.mean()
df['y_pred_price_m2_mean_area'] = global_mean_price_m2 * df.area

In [7]:
mae(df.y_pred_price_m2_mean_area, df.price)

229690.14459799396

In [8]:
df['price_log'] = np.log( df.price )

mean_price_m2_log = np.log( df['price_m2'] ).mean()
y_pred = np.exp(mean_price_m2_log ) * df.area

In [9]:
mae(df.price, y_pred)

223295.69558736696

In [10]:
df_kitchen = df.groupby(['kitchen'])['price'].mean().reset_index()

In [11]:
df_kitchen 

,kitchen,price
0,-1,838825.23
1,oddzielna,544701.64
2,"oddzielna, z wyposażeniem",2590000.00
3,prześwit,519750.00
4,w aneksie,767834.69
5,"w aneksie, w zabudowie",205000.00
6,z oknem,633653.67
7,"z oknem, oddzielna",450375.00
8,"z oknem, oddzielna, z wyposażeniem",382500.00
9,"z oknem, oddzielna, z wyposażeniem, w zabudowie",772800.00


In [13]:
df_kitchen.columns = ['kitchen', 'price_by_kitchen']
df_merged = pd.merge( df, df_kitchen, on='kitchen')

In [14]:
mae(df_merged.price, df_merged.price_by_kitchen)

398812.7793247866

In [15]:
df_kitchen_log = df.groupby('kitchen')['price_log'].mean().reset_index()

In [16]:
df_kitchen_log.columns = ['kitchen', 'price_log_by_kitchen']
%time df_merged = pd.merge( df, df_kitchen_log, on='kitchen')

Wall time: 15.6 ms


In [17]:
y_pred = np.exp( df_merged.price_log_by_kitchen )
mae(df_merged.price, y_pred)

359258.13233915385

In [18]:
keys = ['num_rooms', 'ownership', 'num_floor']
df_grp = df.groupby(keys)['price_log'].mean().reset_index()

df_grp.head()

,num_rooms,ownership,num_floor,price_log
0,-1,-1,-1,13.66
1,-1,-1,1,13.63
2,-1,-1,2,13.38
3,-1,-1,3,13.46
4,-1,-1,4,13.57


In [20]:
new_price_column = 'price_log_by_num_rooms_ownership_num_floor'
df_grp.rename(columns={'price_log': new_price_column}, inplace=True)
df_merged = pd.merge( df, df_grp, on=keys)

In [21]:
y_pred = np.exp( df_merged[new_price_column] )
mae(df_merged.price, y_pred)

240274.28392941243

In [22]:
def make_pred_by_groupping(keys, agg_func=np.mean):

    df_grp = df.groupby(keys)['price_log'].agg(agg_func).reset_index()
    
    new_price_column = 'price_log_by_{}'.format( '_'.join(keys) )
    df_grp.rename(columns={'price_log': new_price_column}, inplace=True)
    df_merged = pd.merge( df, df_grp, on=keys)
    
    y_pred = np.exp( df_merged[new_price_column] )
    return mae(df_merged.price, y_pred)

In [23]:
make_pred_by_groupping(['num_rooms', 'ownership', 'num_floor'])

240274.28392941243

In [25]:
make_pred_by_groupping([ 'kitchen', 'num_rooms', 'ownership', 'num_floor'])

187785.85996643847

In [26]:
make_pred_by_groupping(['parking', 'kitchen', 'num_rooms', 'ownership', 'num_floor'])

95669.11741032728

In [27]:
make_pred_by_groupping(['parking','kitchen', 'loudness', 'num_rooms',  'num_floor', 'area','installations','finishing','location','windows','condition_general','ownership'])

14.044943820582713

In [28]:
pd.cut( df['area'], 5 )

0     (19.537, 112.674]
0    (112.674, 205.348]
0    (205.348, 298.022]
0     (19.537, 112.674]
0     (19.537, 112.674]
            ...        
0     (19.537, 112.674]
0    (112.674, 205.348]
0     (19.537, 112.674]
0     (19.537, 112.674]
0     (19.537, 112.674]
Name: area, Length: 896, dtype: category
Categories (5, interval[float64]): [(19.537, 112.674] < (112.674, 205.348] < (205.348, 298.022] < (298.022, 390.696] < (390.696, 483.37]]

In [29]:
df['area_cut5'] = pd.cut( df['area'], 5).factorize()[0]

In [31]:
make_pred_by_groupping(['parking','kitchen', 'loudness', 'num_rooms',  'num_floor', 'area_cut5','installations','finishing','location','windows','condition_general','ownership'])

17551.13483146101